In [169]:
from IPython.display import display, Math, Latex
import time
import random


#a(x)=3*x^5 - 18*x^4 - 18*x^3 + 20*x^2 - 8*x + 2
#b(x)=-4*x^2 + 6*x - 2

check_time = False
verbose = False
#verbose = True
check_time = True

bit_precision = 40
modular_ring_count = 1

polynom_list = get_random_polynoms(4 , 300)
polynom_list1 = get_random_polynoms(4 , 250)

#polynom_list = get_random_sparse_polynoms(4, 0.1, 300, verbose)
#polynom_list1 = get_random_sparse_polynoms(4, 0.1, 250, verbose)

if check_time:
    complete_start = time.time()
for i in polynom_list:
    for j in polynom_list1:
        if check_time:
            start = time.time()
        compute_ggt_modular_arithmetic(i,j, modular_ring_count, bit_precision, 1, verbose)
        if check_time:
            end = time.time()
            print(end - start)
if check_time:        
    complete_end = time.time()
    print("Time needed")
    print(complete_end - complete_start)





0.14690947532653809
0.14453792572021484
0.1504688262939453
0.17252302169799805
0.1568434238433838
0.14783477783203125
0.1467113494873047
0.14279651641845703
0.14348435401916504
0.14470815658569336
0.15320563316345215
0.15999794006347656
0.1445481777191162
0.1434459686279297
0.14468693733215332
0.14312148094177246
Time needed
2.3891103267669678


In [168]:
def compute_ggt_modular_arithmetic(polynom_a, polynom_b , modular_ring_count, bit_precision, iteration, verbose):
    try:
        
        #Trys to convert coefficients to integers, also makes the higher grade polynom the numerator
        numerator_pol, denominator_pol = prepare_pols(polynom_a , polynom_b)
        
        #to keep track of modular rings they are stored in a dictionary.
        #dictionary key is the ring, value is the polynom tupel
        pol_dict = prepare_dict(numerator_pol, denominator_pol , modular_ring_count, bit_precision)

        #recurisv call, if the structure of the polynom in R[x] is different from the structure of the modular ring 
        #this function will call itself and adds another Modular Ring. 
        #it also keeps also track of iteration and increases precision every 3rd iterration
        if not compute_ggt(pol_dict, verbose):
            iteration +=1
            modular_ring_count += 1
            if iteration % 3 is 0:
                bit_precision *= 2
            compute_ggt_modular_arithmetic(numerator_pol, denominator_pol, modular_ring_count, bit_precision)
            
    except e:
        print('Exception: ' +str(polynom_a) + '\n' + str(polynom_b))
        print(e)

In [155]:
def compute_ggt(pol_dict, verbose):
    try:
        
        #The reference_ring is used to find the real polynomial ring, pop removes this entry from the list
        #it would also be possible to declare another real polynomial ring to find the polynom tupel, 
        #but this requires the bit_precision as parameter. thats why im using the already declared dictionary structure
        real_reference_ring = pol_dict.pop('reference_ring')
        cnt = 1
        
        #Value of the dictionary is a polynom tupel. On [0] is the numerator, [1] is denominator
        #when the polynoms are divided the denominator becomes the numerator, remainder becomes denominator
        #the assignment is done after divide_pol therefroe when this check fires, we're checking for the remainder
        #if it is 0, the currently assigned numerator[1] is the gcd
        while pol_dict[real_reference_ring][1] != 0:
            if verbose:
                print('Iteration: ' + str(cnt))
                
        #to keep track and later compare the remainders, 
        #each of the modular rings will be used as a key to find the corresponding remainder
        #the remainder of each divison will be stored in this dictionary
            remainder_dict = {}
        
        #dividing polynoms, assigning the denominator to numerator and the remainder to denominator
        #divide_pol returns a tupel of polynoms, overwriting the value in pol_dict
        #therefore pol_tupel[1] is the calculated remainder and can be assigned to the 
        #remainder_dict for structure checking
            for ring, pol_tupel in pol_dict.items():
                pol_dict[ring] = divide_pol(pol_tupel[0], pol_tupel[1], verbose)
                remainder_dict[ring] = pol_tupel[1]

        #compares every polynom remainder in a modular ring with the polynom in the real ring
        #if they dont have the same amount of coefficients , the real polynom had a rounding error
        #or the prime number used for the modular ring is a divisor of the inital polynom
        #compute ggt returns false, and the function will be called recursiv 
            real_pol = remainder_dict[real_reference_ring]
            for remainder in remainder_dict.values():
                if not pol_has_same_structure(real_pol, remainder):
                    print(str(real_pol) + 'has not the same structure as' + str(remainder) )
                    return false
            cnt += 1
        if verbose:
            print('GCD is ' + str(pol_dict[real_reference_ring][0]))
        return true
    
    except:
        raise

In [156]:
def divide_pol(numerator_pol, denominator_pol, verbose):
    try:
        #divides polynoms
        #if an exception occurs the numerator, the denominator and the corresponding modular ring is printed
        #also an exception will be raised
        result = numerator_pol.quo_rem(denominator_pol)
        if verbose:
            print_result(numerator_pol, denominator_pol, result[1], numerator_pol.parent())
        return denominator_pol , result[1]
    
    except:
        print('Exception: ' + str(numerator_pol.parent()))
        print('Exception: Dividing '+str(numerator_pol) +' with '+str(denominator_pol))
        raise 

In [157]:
def prepare_pols(pol_a, pol_b):
    #used to convert the coefficients of a polynom to Integers
    #thus random created input polynoms can be created in QQ
    #for now, this implemantation seems terrible, coefficients get way to big
    
    #pol_a, pol_b = make_coeff_integer(pol_a, pol_b)
    
    
    #makes the higher grade polynom the numerator
    if len(pol_a.coefficients()) < len(pol_b.coefficients()):
        return pol_b , pol_a
    return pol_a, pol_b

In [158]:
def prepare_dict(pol_a, pol_b, modular_ring_count, bit_precision ):
    
    #to keep track of modular rings they are stored in a dictionary.
    #the dictionary key is the ring,the value is the polynom tupel
    #first entry is the Real polynomial ring, it will later be used to find the the corresponding polynoms
    #it would also be possible to declare another real polynomial ring to find the polynom tupel, 
    #but this requires the bit_precision as parameter. thats why im using the already declared dictionary structure
    #the first entry will later be poped
    #second entry is a tupel of the numerator and denominator polynom, key is the Real polynomial Ring
    real_pol_ring = PolynomialRing(RealField(bit_precision), 'x')
    
    pol_a = real_pol_ring(pol_a)
    pol_b = real_pol_ring(pol_b)
    pol_dict = {}
    
    pol_dict['reference_ring'] = real_pol_ring
    pol_dict[real_pol_ring] = [pol_a, pol_b]
    
    #adds the specified number of modular rings, primenumbers are choosen at random
    #like before the dictionary value is a tupel of polynoms, the key is the corresponding modular ring
    
    for x in range(modular_ring_count):
        primeRing = PolynomialRing(FiniteField(random_prime(2^24,False,2^22)), 'x')
        pol_a = primeRing(pol_a)
        pol_b = primeRing(pol_b)
        pol_dict[primeRing] = [pol_a , pol_b]
    
    return pol_dict

In [159]:
def get_random_polynoms(count, deg):
    ran_pol_list = []
    for i in range(count): 
        ran_pol_list.append(ZZ[x].random_element(degree=deg))
    return ran_pol_list
        

In [160]:
def print_result(numerator , demnominator , remainder,  ring):
    
    #formatting for verbose mode
    #because the display functions also prints 0 coefficients it only pretty prints low degree polynoms 
    if len(numerator.coefficients()) < 10:
        display(Math(latex(ring)))
        print('Numerator')
        display(Math(latex(numerator)))
        print('Denominator' )
        display(Math(latex(demnominator)))
        print('Remainder ')
        display(Math(latex(remainder)))
        print('\n')
    else:
        display(Math(latex(ring)))
        print('Numerator')
        print(numerator)
        print('Denominator' )
        print(demnominator)
        print('Remainder ')
        print(remainder)
        print('\n')      


In [161]:
def pol_has_same_structure(pol_a, pol_b):
    #check length of coefficients, 0's are also included
    if len(pol_a.coefficients()) != len(pol_b.coefficients()):
        false
    return true
    

In [162]:
def get_random_sparse_polynoms(count, density, degree, verbose):
    ran_sparse_list = []
    
    for i in range(count):
        #creates a degree sized list of 0  
        random_coefficients = [0] * (degree + 1)
        
        #eyeballs the amount of coefficients
        amount_coefs = round(degree * density)
        
        #makes sure the polynom has the specified degree
        random_coefficients[degree] = random.randint(0,10)

        for j in range(amount_coefs):
            #eyeballs the coefficients assignments
            random_coefficients[random.randint(0,degree)] = random.randint(0,10)
        
        #horner scheme to create the polynom
        p = 0
        for j in reversed(random_coefficients):
            p = p*x+j
        
        if verbose:
            pr = PolynomialRing(ZZ, 'x')
            p = pr(p)
            print(p)
            
        ran_sparse_list.append(p)

    return ran_sparse_list
    

In [152]:
def make_coeff_integer(pol):
    #its a work in progress
    
    coeffs = []
    coeffs2 = []
    
    for coeff in pol.coefficients():
        coeffs.append(Rational(coeff[0]))
        
    p = 0
    denum = 1
    for i in (coeffs):
        denum *= i.denominator()
        
    for i in (coeffs):
        coeffs2.append(i*denum)

    for i in reversed(coeffs2):
          p = p*x+i


    return p